In [1]:
import cv2
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import time as tm

In [2]:
train = "train"
split_img = "split_img"
os.system('rm -rf split_img')
try:
    os.mkdir(split_img)
except:
    garbage = 0


for i in range(65,91):
    try:
        os.mkdir(split_img + "/" + chr(i))
    except: 
        garbage = 0
       
captchas = glob.glob(os.path.join(train, "*"))


# count = np.zeros([256])
# captcha = "train/QDMK.png"
# img = cv2.imread(captcha,0)

# cv2.imshow('image',img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# for i in range(len(img)) :  
#     for j in range(len(img[i])) :  
#         count[img[i][j]] += 1
# sort_count = np.sort(count)
# sort_ind = np.argsort(count)[-20:-1]
# print(sort_ind)

# for i in range(len(img)) :  
#     for j in range(len(img[i])) :  
#         flag = 0
#         for k in sort_ind :
#             if(img[i][j] == k) :
#                 flag = 1
#                 break
#         if(flag == 1):
#             img[i][j] = 0
#         else:
#             img[i][j] = 255
            
# cv2.imshow('image',img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
tic = tm.perf_counter()

count = 0
for (i, captcha) in enumerate(captchas):
#     captcha = "train/AVR.png"
#     print(captcha,i)
    img = cv2.imread(captcha)   #read image as grayscale
    
    first_px = img[0,0]
    img[np.where(img == first_px)] = 0
    
    kernel = np.ones((6,6),np.uint8)
    erosion = cv2.erode(img, kernel, iterations = 1)  
    img = cv2.morphologyEx(erosion, cv2.MORPH_OPEN, kernel)
    
    img[np.where(img == 0)] = 255
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    img[np.where(img != 255)] = 0
    
    crop_s = []
    crop_e = []
    
    start = 0
    end = 0
    flag = 1
    
    
    img = np.transpose(img)
    
    #print(len(img))
    
    num_px = 0 # number of pixels
    
    for i in range(len(img)):
        if  flag and np.sum(img[i]) != 255*(len(img[i])) :
            start = i
            flag = 0
        elif flag == 0 and np.sum(img[i]) == 255*(len(img[i])) :
            flag = 1
            mid = int((i + start)/2)
            if i - start > 25 and np.sum(img[mid])<(255*(len(img[mid])-10)):
                crop_s.append(start)
                crop_e.append(i)
    #print(crop)
    
    img = np.transpose(img)
#     cv2.imshow('image',img)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    filename = os.path.basename(captcha)
    captcha_text = os.path.splitext(filename)[0]
    if (len(crop_s)-len(captcha_text)) != 0 :
        print(captcha_text)
        print(len(crop_s))
        print(len(captcha_text))
    for i in range(len(crop_s)):
        if i == len(captcha_text):
            break
        x_ = crop_s[i]
        y_ = crop_e[i]
        crop_img = img[:,x_-5:y_+5]
        save_path = os.path.join(split_img, captcha_text[i])
        p = os.path.join(save_path, "{}.png".format(str(count)))
        cv2.imwrite(p, crop_img)
        count += 1
        
#     break

toc = tm.perf_counter()
print(toc-tic)

62.265679422001995


In [3]:
# training a model 
X = []
y = []
solved_letters = glob.glob(os.path.join("split_img/*", "*"))
for i,img in enumerate(solved_letters):
    try:
        txt = img.split('/')[1]
#         yi =np.zeros(26)
#         yi[ord(txt)-65] =1
#             print(img)
        yi = ord(txt)-65
        img = cv2.imread(img,0)
        dim = (28,28)
        img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
        img = img.reshape(-1)
        X.append(np.array(img))
        y.append(yi)
#         print(img.shape)
#         break
    except:
        print("Not a image")

In [4]:
X = np.array(X)
y = np.array(y)
print(X.shape)
print(y.shape)

(7005, 784)
(7005,)


In [5]:
X_train = X.reshape(X.shape[0], 28, 28, 1)
X_train = X_train.astype('float32')
# Normalizing the RGB codes by dividing it to the max RGB value.
X_train /= 255

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
# Creating a Sequential Model and adding the layers
input_shape = (28, 28, 1)
model = Sequential()
model.add(Conv2D(28, padding="same", kernel_size=(5,5), input_shape=input_shape, activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))

model.add(Conv2D(50, (5, 5), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Flatten()) # Flattening the 2D arrays for fully connected layers
model.add(Dense(100, activation=tf.nn.relu))
#model.add(Dropout(0.2))
model.add(Dense(26,activation=tf.nn.softmax))
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 28)        728       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 28)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 50)        35050     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 50)          0         
_________________________________________________________________
flatten (Flatten)            (None, 2450)              0         
_________________________________________________________________
dense (Dense)                (None, 100)               245100    
_________________________________________

In [7]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
model.fit( X_train, y, epochs=10)

Train on 7005 samples
Epoch 1/10
7005/7005 [==============================] - 12s 2ms/sample - loss: 0.7759 - acc: 0.7863
Epoch 2/10
7005/7005 [==============================] - 10s 1ms/sample - loss: 0.0293 - acc: 0.9919
Epoch 3/10
7005/7005 [==============================] - 11s 2ms/sample - loss: 0.0152 - acc: 0.9963
Epoch 4/10
7005/7005 [==============================] - 9s 1ms/sample - loss: 0.0057 - acc: 0.9986
Epoch 5/10
7005/7005 [==============================] - 10s 1ms/sample - loss: 0.0121 - acc: 0.9963
Epoch 6/10
7005/7005 [==============================] - 10s 1ms/sample - loss: 0.0040 - acc: 0.9989
Epoch 7/10
7005/7005 [==============================] - 9s 1ms/sample - loss: 6.7730e-04 - acc: 0.9997
Epoch 8/10
7005/7005 [==============================] - 10s 1ms/sample - loss: 0.0110 - acc: 0.9973
Epoch 9/10
7005/7005 [==============================] - 10s 1ms/sample - loss: 3.7981e-04 - acc: 1.0000
Epoch 10/10
7005/7005 [==============================] - 10s 1ms/sample 

In [8]:
tic = tm.perf_counter()
model.evaluate(X_train, y)
toc = tm.perf_counter()
print( "Total time taken is %.6f seconds " % (toc - tic) )

7005/7005 [==============================] - 4s 579us/sample - loss: 7.2170e-05 - acc: 1.0000
Total time taken is 4.067054 seconds 


In [9]:
model.save("model.h5")